In [1]:
import bokit as bk

# NNF

In [2]:
vs = bk.VarSpace()
e = vs.parse_expression("!(A & B | C & !(!D | B))", extend=True)
e

!(__0__ & __1__ | __2__ & __3__ & !__1__)

In [3]:
e2 = vs.parse_expression("A & (B | C)")
ne2 = ~e2

In [4]:
vs.display( ne2.nnf() )

'!A | !B & !C'

In [5]:
vs.display( e.nnf() )

'(!A | !B) & (!C | !D | B)'

# Complexity analysis

In [6]:
vs = bk.VarSpace()
e = vs.parse_expression("(A & B & C) | (D & E & F) | (G & H & I) | (J & K & L)", extend=True)
ne = ~e

ne.estimate_complexity()

ExprComplexity { score: Some(81), atoms: 12, depth: 4 }

In [7]:
# this function is too simple to be decomposed using the default penalty
dc,rep = ne.decompose()
print(rep)
dc

DecompositionReport { root_score: 81, sum_sub_score: 0 }


!(__0__ & __1__ & __2__ | __3__ & __4__ & __5__ | __6__ & __7__ & __8__ | __9__ & __10__ & __11__)

In [8]:
# By selecting a lower penalty, we can decompose it
dc,rep = ne.decompose(50)
print(rep)
dc

DecompositionReport { root_score: 3, sum_sub_score: 27 }


Expanded __12__ & (!__9__ | !__10__ | !__11__)
  _12_ <- (!__0__ | !__1__ | !__2__) & (!__3__ | !__4__ | !__5__) & (!__6__ | !__7__ | !__8__)

In [9]:
pe = bk.Primes(e)
epe = bk.Expr(pe)
epe.estimate_complexity()

ExprComplexity { score: Some(4), atoms: 12, depth: 4 }

In [10]:
nepe = ~epe
nepe.estimate_complexity()

ExprComplexity { score: Some(81), atoms: 12, depth: 4 }

In [11]:
# Complex rule taken from https://www.nature.com/articles/s41467-019-08903-w
cplx = "((R43 & R939 & S1166) | ((S45 | S44) & (S43 | S42) & (S47 | S46) & (S431 | S1167) & (S575 | S326 | S1169) & (S1166 | S47) & (S1173 | S1172) & (S1168 | S471) & (S1171 | S1170) & ((R43 & (R939 | S46 | ((S45 | S44) & (S43 | S42) & (S47 | S46) & ((S47 & R44) | (S46 & !((R43 & S1166 & S46)))))) & (S1166 | ((S431 | S1167) & (S575 | S326 | S1169) & (S1166 | S47) & (S1173 | S1172) & (S1168 | S471) & (S1171 | S1170) & ((S47 & R44) | (S1166 & !((R43 & S1166 & S46))))))) | (S47 & !((S47 & R44))))))"
spl = "R939 & S1166 | (S45 | S44) & (S43 | S42)"

In [12]:
elx = vs.parse_expression(cplx, extend=True)
len(elx.regulators()), elx.estimate_complexity()

(21, ExprComplexity { score: Some(10054657), atoms: 60, depth: 12 })

In [13]:
(~elx).estimate_complexity()

ExprComplexity { score: Some(507), atoms: 60, depth: 12 }

In [14]:
npr = bk.Primes(~elx)
npr.len()

25

In [15]:
elx.estimate_complexity()

ExprComplexity { score: Some(10054657), atoms: 60, depth: 12 }

In [16]:
%time pr = bk.Primes(elx)

CPU times: user 207 ms, sys: 0 ns, total: 207 ms
Wall time: 208 ms


In [17]:
pr.len()

577

In [18]:
nepr = bk.Expr(npr)
nepr.regulators().len()

21

In [19]:
vs.display(elx)

'R43 & R939 & S1166 | (S45 | S44) & (S43 | S42) & (S47 | S46) & (S431 | S1167) & (S575 | S326 | S1169) & (S1166 | S47) & (S1173 | S1172) & (S1168 | S471) & (S1171 | S1170) & (R43 & (R939 | S46 | (S45 | S44) & (S43 | S42) & (S47 | S46) & (S47 & R44 | S46 & (!R43 | !S1166 | !S46))) & (S1166 | (S431 | S1167) & (S575 | S326 | S1169) & (S1166 | S47) & (S1173 | S1172) & (S1168 | S471) & (S1171 | S1170) & (S47 & R44 | S1166 & (!R43 | !S1166 | !S46))) | S47 & (!S47 | !R44))'

In [20]:
vs.display(nepr)

'!R43 & !S45 & !S44 | !R939 & !S45 & !S44 | !S1166 & !S45 & !S44 | !R43 & !S43 & !S42 | !R939 & !S43 & !S42 | !S1166 & !S43 & !S42 | !S1166 & !S1171 & !S1170 | !R939 & !S1171 & !S1170 | !S1166 & !S575 & !S326 & !S1169 | !R43 & !S431 & !S1167 | !R939 & !S431 & !S1167 | !S1166 & !S431 & !S1167 | !R43 & !S575 & !S326 & !S1169 | !R939 & !S575 & !S326 & !S1169 | !R43 & !S1171 & !S1170 | !R43 & !S1173 & !S1172 | !R939 & !S1173 & !S1172 | !S1166 & !S1173 & !S1172 | !R43 & !S1168 & !S471 | !R939 & !S1168 & !S471 | !S1166 & !S1168 & !S471 | !S1166 & !S47 | !R939 & !S47 & !S46 | !R43 & !S47 | R44 & !R43'

In [21]:
nepr.estimate_complexity()

ExprComplexity { score: Some(25), atoms: 75, depth: 25 }

In [22]:
epr = ~ nepr

In [23]:
epr.estimate_complexity()

ExprComplexity { score: Some(595077871104), atoms: 75, depth: 25 }

In [24]:
esl = vs.parse_expression(spl, extend=True)
esl.estimate_complexity()

ExprComplexity { score: Some(5), atoms: 6, depth: 3 }

In [25]:
epr.estimate_complexity()

ExprComplexity { score: Some(595077871104), atoms: 75, depth: 25 }

In [26]:
dc,rep = epr.decompose()
print(rep)
dc

DecompositionReport { root_score: 72, sum_sub_score: 495 }


Expanded __4__ & (__14__ | __28__ | __29__) & (__14__ | __19__) & (__13__ | __19__ | __20__) & (__12__ | __19__) & (!__32__ | __12__)
  _3_ <- __2__ & (__12__ | __23__ | __24__ | __25__) & (__13__ | __23__ | __24__ | __25__) & (__12__ | __30__ | __31__) & (__12__ | __26__ | __27__)
  _0_ <- (__12__ | __15__ | __16__) & (__13__ | __15__ | __16__) & (__14__ | __15__ | __16__) & (__12__ | __17__ | __18__)
  _1_ <- __0__ & (__13__ | __17__ | __18__) & (__14__ | __17__ | __18__) & (__14__ | __30__ | __31__) & (__13__ | __30__ | __31__)
  _4_ <- __3__ & (__13__ | __26__ | __27__) & (__14__ | __26__ | __27__) & (__12__ | __28__ | __29__) & (__13__ | __28__ | __29__)
  _2_ <- __1__ & (__14__ | __23__ | __24__ | __25__) & (__12__ | __21__ | __22__) & (__13__ | __21__ | __22__) & (__14__ | __21__ | __22__)

In [27]:
dc.count_prime_implicants()

54